In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import scipy.stats as st
import flask as Flask

In [2]:
# Read in csv file and display data
pollution = pd.read_csv("data\global air pollutiondataset.csv")
pollution.head()

,City_ID,Country,City,AQI_Value,AQI_Category,CO_AQI_Value,CO_AQI_Category,Ozone_AQI_Value,Ozone_AQI_Category,NO2_AQI_Value,NO2_AQI_Category,PM2.5_AQI_Value,PM2.5_AQI_Category
0,1,Russian Federation,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate
1,2,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good
2,3,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate
3,4,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good
4,5,France,Punaauia,22,Good,0,Good,22,Good,0,Good,6,Good


In [3]:
# summary of the global air pollution dataset
pollution.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23463 entries, 0 to 23462
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   City_ID             23463 non-null  int64 
 1   Country             23036 non-null  object
 2   City                23462 non-null  object
 3   AQI_Value           23463 non-null  int64 
 4   AQI_Category        23463 non-null  object
 5   CO_AQI_Value        23463 non-null  int64 
 6   CO_AQI_Category     23463 non-null  object
 7   Ozone_AQI_Value     23463 non-null  int64 
 8   Ozone_AQI_Category  23463 non-null  object
 9   NO2_AQI_Value       23463 non-null  int64 
 10  NO2_AQI_Category    23463 non-null  object
 11  PM2.5_AQI_Value     23463 non-null  int64 
 12  PM2.5_AQI_Category  23463 non-null  object
dtypes: int64(6), object(7)
memory usage: 2.3+ MB


In [4]:
# See all the countries
country = pollution["Country"].unique()
print(country)

['Russian Federation' 'Brazil' 'Italy' 'Poland' 'France'
 'United States of America' 'Germany' 'Belgium' 'Egypt' 'China'
 'Netherlands' 'India' 'Pakistan' 'Republic of North Macedonia' 'Colombia'
 'Romania' 'Indonesia' 'Finland' 'South Africa'
 'United Kingdom of Great Britain and Northern Ireland'
 'United Republic of Tanzania' 'Haiti' 'Somalia' 'Philippines' 'Latvia'
 'Chad' 'New Zealand' 'Tunisia' 'Viet Nam' 'Iran (Islamic Republic of)'
 'Mexico' 'Japan' 'El Salvador' 'Bulgaria' 'Nigeria' 'South Sudan'
 'Guatemala' 'Ireland' 'Turkey' 'Peru' 'Democratic Republic of the Congo'
 'Canada' 'Switzerland' 'Denmark' 'Cameroon' 'Australia' 'Portugal'
 "Côte d'Ivoire" 'Sweden' 'Ethiopia' 'Thailand' 'Hungary' 'Kazakhstan'
 'Israel' 'Spain' 'Myanmar' 'Papua New Guinea' 'Madagascar' 'Lithuania'
 'Ghana' 'Azerbaijan' 'Armenia' 'Ukraine' 'Malaysia' 'Serbia' 'Slovakia'
 'Gambia' 'Ecuador' 'Bosnia and Herzegovina' 'Czechia' 'Argentina'
 'Dominican Republic' 'Guinea' 'Bolivia (Plurinational State of)

In [5]:
# Find out how many countries are in the dataset
print(len(country))

176


In [6]:
# Find out how many Cities are in the dataset
city = pollution["City"].unique()
print(len(city))

23463


In [ ]:
pollution.dropna

In [ ]:
#group Cities with AQI Values


# Find out the cities with the most overall pollution
# AQI_values_per_city = pollution["AQI Value"].sort_values(ascending = False)
# print(AQI_values_per_city)

In [ ]:
# Read in csv file (part 2)
population = pd.read_csv("data\air_index.csv")
population.head()
